In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

# get ANOVA table as R like output
import statsmodels.api as sm
from statsmodels.formula.api import ols

with open('res_sub_tab.txt', 'w') as res_sub_tab:
    res_sub_tab.write('Tabelas Latex para os resultados dos métodos subjetivos')
    res_sub_tab.write('\n')
    res_sub_tab.close()




# Leitura e organização dos dados

## Leitura dos dados brutos

In [2]:
df = pd.read_excel("Questionários pesquisa.xlsx")
participantes = df.loc[:,['Participante:','Idade:','Sexo:','Como deficiente visual você se identifica como:']]
participantes.set_index('Participante:', drop = True, inplace = True)
df_resto_1 = df.drop(['Idade:','Sexo:','Como deficiente visual você se identifica como:'],axis = 1)

## Separação dos dados

### Ordem dos cenários

In [3]:
ordemCenarios = df_resto_1.iloc[:,range(1,7)]
#ordemCenarios = ordemCenarios.drop(ordemCenarios.columns[range(1,4)], axis = 1)
ordemCenarios.set_index(keys = "Participante:", inplace = True)
df_resto_2 = df_resto_1.drop(ordemCenarios.columns,axis = 1)


### Nasa

In [4]:
cenas = list(range(1,6))
nomeCenas = ['Base', 'Audio', 'Haptic Belt', 'Virtual Cane', 'Mixture']
etapas = [1,2]
dimensoesNasa = ['Demanda Mental','Demanda física', 'Demanda Temporal', 'Perfomance', 'Esforço', 'Frustação']
dimensoesNasas_todas = ['Participante:']
for c in cenas:
    for e in etapas:
        for d in dimensoesNasa:
            nome = str(c) + '.' + str(e) + ') '+ d
            dimensoesNasas_todas.append(nome)

dfNasa = df_resto_2.loc[:,dimensoesNasas_todas]
dfNasa.set_index('Participante:',inplace = True)
df_resto_3 = df_resto_2.drop(dfNasa.columns, axis = 1)
# dfNasa.head()


### Sagat

In [5]:
perguntasSagat = ['Existe algum objeto próximo de você?', 'Sinalize onde o objeto está', 'Existe alguém perto de você?', 'Sinaliza onde está pessoa esta', 
'Você percebeu alguma fonte de som característica do lugar onde você se encontra?', 'Sinalize de onde vem o som', 'Sinalize em que direção está a recepcionista', 
'Sinalize em que direção está a saída', 'Qual a distância você imagina que existe entre a mesa da recepção e qual direção ela está?',	
'Qual a distância você imagina que existe entre a saída?']

perguntasSagat_todas = ['Participante:'] 
for c in cenas:
    for e in etapas:
        for p in perguntasSagat:
            nome = str(c) + '.' + str(e) + ') '+ p
            perguntasSagat_todas.append(nome)

dfSagat = df_resto_3.loc[:,perguntasSagat_todas]
dfSagat.set_index('Participante:',inplace = True)
df_resto_4 = df_resto_3.drop(dfSagat.columns, axis = 1)
#dfSagat.head()

### Questionário dos métodos de guiagem

In [6]:
dfQuestionario = df_resto_4.set_index('Participante:').drop('Carimbo de data/hora', axis = 1 )
#dfQuestionario.head()

## Reorganizar os dados

### Transformar dados qualitativos em quantitativos

Cego ficou definido como número 0
Vidente ficou definido como número 1

Masculino ficou definido como número 0
Feminino ficou definido como número 1

In [7]:
participantes.insert(2,'Sexo 0/1', pd.factorize(participantes.iloc[:,1])[0])
participantes.insert(4,'Deficiência',pd.factorize(participantes.iloc[:,3])[0])
participantes

,Idade:,Sexo:,Sexo 0/1,Como deficiente visual você se identifica como:,Deficiência
Participante:,,,,,
001C,56,Masculino,0,Cego,0
001,22,Masculino,0,Não sou deficiente visual,1


### Nasa

In [8]:
nasa = []
for p in range(len(participantes)):
    nasaPorCena = []
    for c in range(len(cenas)):
        nasaPorEtapa = []
        for e in range(len(etapas)):
            nasaEtapa = []
            for d in range(len(dimensoesNasa)):
                #print(dfNasa.iloc[d+e*6+c*12])
                nasaEtapa.append(dfNasa.iloc[p,d+e*6+c*12])
            nasaPorEtapa.append(nasaEtapa)
        nasaPorCena.append(nasaPorEtapa)
    nasa.append(nasaPorCena)
#nasa

### Sagat

In [9]:
sagat = []
for p in range(len(participantes)):
    sagatPorCena = []
    for c in range(len(cenas)):
        sagatPorEtapa = []
        for e in range(len(etapas)):
            sagatEtapa = []
            for d in range(len(perguntasSagat)):
                #print(dfSagat.iloc[d+e*10+c*20])
                sagatEtapa.append(dfSagat.iloc[p,d+e*10+c*20])
            sagatPorEtapa.append(sagatEtapa)
        sagatPorCena.append(sagatPorEtapa)
    sagat.append(sagatPorCena)
#sagat

### Questionários dos métodos de guiagem

In [10]:
perguntasQuestionario = [7,6,7,20]

questionarios = []
for p in range(len(participantes)):
    questionariosPorCena = []
    for c in range(len(cenas)-1):
        questionarioCena = []
        for d in range(perguntasQuestionario[c]):
            #print(dfQuestionario.iloc[d+sum(perguntasQuestionario[:c])])
            questionarioCena.append(dfQuestionario.iloc[p,d+sum(perguntasQuestionario[:c])])
        questionariosPorCena.append(questionarioCena)
    questionarios.append(questionariosPorCena)
questionarios

[[[2,
   1,
   1,
   6,
   7,
   3,
   'Sim. Precisão. Exmeplo a Mesa está a 2m. Falta a orientação da mesa.'],
  [5, 1, 5, 5, 3, 'Sim. Faltou perceber as paredes'],
  [5,
   3,
   6,
   1,
   1,
   'Sim. Falta informação sobre a forma do objeto',
   'Bengala, pq ela informa a parede'],
  [4,
   3,
   1,
   1,
   1,
   1,
   6,
   1,
   1,
   6,
   1,
   1,
   3,
   5,
   3,
   5,
   1,
   2,
   'Sim, falta a forma do objeto',
   'Audio não foi necessário']],
 [[3, 4, 7, 6, 7, 3, 'Não'],
  [2, 2, 5, 5, 3, 'Não'],
  [5, 1, 2, 4, 3, 'Não', 'Bengala, por ser mais precisa'],
  [4, 5, 6, 5, 5, 3, 3, 2, 4, 2, 3, 7, 5, 6, 0, 3, 1, 2, 'Não', 'Cinto']]]

# Separar e definir os valores para análise

## Nasa

### Demanda Mental

In [11]:
mdNasa = []
for p in range(len(participantes)):
    mdNasaPorCena = []
    for c in range(len(cenas)):
        mdNasaPorEtapa = []
        for e in range(len(etapas)):
            #print(nasa[p][c][e][0])
            mdNasaPorEtapa.append(np.mean(nasa[p][c][e][0]))
        mdNasaPorCena.append(mdNasaPorEtapa)
    mdNasa.append(mdNasaPorCena)
#mdNasa

### Média Demanda Mental (por cenários, por participante)

In [12]:
mdNasa_media_cen_par = []
for p in range(len(participantes)):
    mdNasaPorCena_media_cen_par = []
    for c in range(len(cenas)):
        #print(mdNasa[p][c][:])
        #print(np.mean(mdNasa[p][c][:]))
        mdNasaPorCena_media_cen_par.append(np.mean(mdNasa[p][c][:]))
    mdNasa_media_cen_par.append(mdNasaPorCena_media_cen_par)
mdNasa_media_cen_par = pd.DataFrame(mdNasa_media_cen_par, columns=nomeCenas, index = participantes.index)
mdNasa_media_cen_par = pd.concat([mdNasa_media_cen_par,participantes.loc[:,'Deficiência']],axis = 1)
mdNasa_media_cen_par.rename(columns = {'Deficiência' : 'Visual Impairity'}, inplace=True) 
mdNasa_media_cen_par.index.names = ['Participant']
mdNasa_media_cen_par

,Base,Audio,Haptic Belt,Virtual Cane,Mixture,Visual Impairity
Participant,,,,,,
001C,2.0,12.0,12.0,2.5,6.0,0
001,6.0,12.5,12.0,5.0,9.5,1


In [13]:
tabela = pd.DataFrame.to_latex(mdNasa_media_cen_par, label = 'tab:media_deman_mental_cen_part', caption = 'Mental demand average grouped by guidance method and participant', position = 'htb')
print(tabela)

with open('res_sub_tab.txt', 'a') as res_sub_tab:
    res_sub_tab.write('\n' +'\n' +'Mental demand average grouped by participant and guidance method' + '\n')
    res_sub_tab.write('\n' + tabela + '\n')

tabela = 0

\begin{table}[htb]
\centering
\caption{Mental demand average grouped by guidance method and participant}
\label{tab:media_deman_mental_cen_part}
\begin{tabular}{lrrrrrr}
\toprule
{} &  Base &  Audio &  Haptic Belt &  Virtual Cane &  Mixture &  Visual Impairity \\
Participant &       &        &              &               &          &                   \\
\midrule
001C        &   2.0 &   12.0 &         12.0 &           2.5 &      6.0 &                 0 \\
001         &   6.0 &   12.5 &         12.0 &           5.0 &      9.5 &                 1 \\
\bottomrule
\end{tabular}
\end{table}



### Média Demanda Mental (por cenários, por deficiencia)

In [14]:
mdNasa_media_cen_def = mdNasa_media_cen_par.groupby(by = 'Visual Impairity').mean()
mdNasa_media_cen_def

,Base,Audio,Haptic Belt,Virtual Cane,Mixture
Visual Impairity,,,,,
0,2.0,12.0,12.0,2.5,6.0
1,6.0,12.5,12.0,5.0,9.5


In [15]:
tabela = pd.DataFrame.to_latex(mdNasa_media_cen_def, label = 'tab:media_deman_mental_cen_def', caption = 'Mental demand average grouped by participant and visual imparity', position = 'htb')
print(tabela)

with open('res_sub_tab.txt', 'a') as res_sub_tab:
    res_sub_tab.write('\n' +'\n' +'Mental demand average grouped by participant and visual imparity'+ '\n')
    res_sub_tab.write('\n' + tabela + '\n')

tabela = 0

\begin{table}[htb]
\centering
\caption{Mental demand average grouped by participant and visual imparity}
\label{tab:media_deman_mental_cen_def}
\begin{tabular}{lrrrrr}
\toprule
{} &  Base &  Audio &  Haptic Belt &  Virtual Cane &  Mixture \\
Visual Impairity &       &        &              &               &          \\
\midrule
0                &   2.0 &   12.0 &         12.0 &           2.5 &      6.0 \\
1                &   6.0 &   12.5 &         12.0 &           5.0 &      9.5 \\
\bottomrule
\end{tabular}
\end{table}



### Carga Mental

In [16]:
cdNasa = []
for p in range(len(participantes)):
    cdNasaPorCena = []
    for c in range(len(cenas)):
        cdNasaPorEtapa = []
        for e in range(len(etapas)):
            #print(nasa[p][c][e][:])
            cdNasaPorEtapa.append(np.mean(nasa[p][c][e][:]))
        cdNasaPorCena.append(cdNasaPorEtapa)
    cdNasa.append(cdNasaPorCena)
#cdNasa

### Média Carga Mental (por cenários, por participante)

In [17]:
cdNasa_media_cen_par = []
for p in range(len(participantes)):
    cdNasaPorCena_media_cen_par = []
    for c in range(len(cenas)):
        #print(cdNasa[p][c][:])
        #print(np.mean(cdNasa[p][c][:]))
        cdNasaPorCena_media_cen_par.append(np.mean(cdNasa[p][c][:]))
    cdNasa_media_cen_par.append(cdNasaPorCena_media_cen_par)
cdNasa_media_cen_par = pd.DataFrame(cdNasa_media_cen_par, columns=nomeCenas, index = participantes.index)
cdNasa_media_cen_par = pd.concat([cdNasa_media_cen_par,participantes.loc[:,'Deficiência']],axis = 1)
cdNasa_media_cen_par.rename(columns = {'Deficiência' : 'Visual Impairity'}, inplace=True) 
cdNasa_media_cen_par.index.names = ['Participant']
cdNasa_media_cen_par

,Base,Audio,Haptic Belt,Virtual Cane,Mixture,Visual Impairity
Participant,,,,,,
001C,4.500000,7.750000,7.75,4.833333,6.250000,0
001,7.916667,10.583333,14.75,6.916667,9.166667,1


In [18]:
tabela = pd.DataFrame.to_latex(cdNasa_media_cen_par, label = 'tab:media_carg_mental_cen_part', caption = 'Mental workload average grouped by participant and guidance method', position = 'htb')
print(tabela)

with open('res_sub_tab.txt', 'a') as res_sub_tab:
    res_sub_tab.write('\n' +'\n' + 'Mental workload average grouped by participant and guidance method'+ '\n')
    res_sub_tab.write('\n' + tabela + '\n')

tabela = 0

\begin{table}[htb]
\centering
\caption{Mental workload average grouped by participant and guidance method}
\label{tab:media_carg_mental_cen_part}
\begin{tabular}{lrrrrrr}
\toprule
{} &      Base &      Audio &  Haptic Belt &  Virtual Cane &   Mixture &  Visual Impairity \\
Participant &           &            &              &               &           &                   \\
\midrule
001C        &  4.500000 &   7.750000 &         7.75 &      4.833333 &  6.250000 &                 0 \\
001         &  7.916667 &  10.583333 &        14.75 &      6.916667 &  9.166667 &                 1 \\
\bottomrule
\end{tabular}
\end{table}



### Média Carga Mental (por cenários, por deficiencia)

In [19]:
cdNasa_media_cen_def  = cdNasa_media_cen_par.groupby(by = 'Visual Impairity').mean()
cdNasa_media_cen_def

,Base,Audio,Haptic Belt,Virtual Cane,Mixture
Visual Impairity,,,,,
0,4.500000,7.750000,7.75,4.833333,6.250000
1,7.916667,10.583333,14.75,6.916667,9.166667


## Sagat

### Pontuação global Sagat

In [20]:
saSagat = []
for p in range(len(participantes)):
    saSagatPorCena = []
    for c in range(len(cenas)):
        saSagatPorEtapa = []
        for e in range(len(etapas)):
            #print(sagat[p][c][e][:])
            saSagatPorEtapa.append(np.mean(sagat[p][c][e][:]))
        saSagatPorCena.append(saSagatPorEtapa)
    saSagat.append(saSagatPorCena)
#saSagat

### Média pontuação global Sagat (por cenários, por participante)

In [21]:
saSagat_media_cen_par = []
for p in range(len(participantes)):
    saSagatPorCena_media_cen_par = []
    for c in range(len(cenas)):
        #print(saSagat[p][c][:])
        #print(np.mean(saSagat[p][c][:]))
        saSagatPorCena_media_cen_par.append(np.mean(saSagat[p][c][:]))
    saSagat_media_cen_par.append(saSagatPorCena_media_cen_par)
saSagat_media_cen_par = pd.DataFrame(saSagat_media_cen_par, columns=nomeCenas, index = participantes.index)
saSagat_media_cen_par = pd.concat([saSagat_media_cen_par,participantes.loc[:,'Deficiência']],axis = 1)
saSagat_media_cen_par.rename(columns = {'Deficiência' : 'Visual Impairity'}, inplace=True) 
saSagat_media_cen_par.index.names = ['Participant']
saSagat_media_cen_par

,Base,Audio,Haptic Belt,Virtual Cane,Mixture,Visual Impairity
Participant,,,,,,
001C,0.575,0.6955,0.675,0.500,0.4500,0
001,1.000,0.5750,0.458,0.483,0.6875,1


### Média pontuação global Sagat (por cenários, por deficiencia)

In [22]:
saSagat_media_cen_def = saSagat_media_cen_par.groupby(by = 'Visual Impairity').mean()
saSagat_media_cen_def

,Base,Audio,Haptic Belt,Virtual Cane,Mixture
Visual Impairity,,,,,
0,0.575,0.6955,0.675,0.500,0.4500
1,1.000,0.5750,0.458,0.483,0.6875


## Questionários sobre os métodos

### Pontuação global guiagem

In [23]:
pgQuestionario_media = []
for p in range(len(participantes)):
    pgQuestionarioPorCena = []
    for c in range(len(cenas)-1):
        pgQuestionarioCena = 0
        if (c == 2 or c == 3):
            #print(questionarios[p][c][:len(questionarios[p][c])-2])
            pgQuestionarioCena = np.mean(np.mean(questionarios[p][c][:len(questionarios[p][c])-2]))
        else:
            #print(questionarios[p][c][:len(questionarios[p][c])-1])
            pgQuestionarioCena = np.mean(np.mean(questionarios[p][c][:len(questionarios[p][c])-1]))
        pgQuestionarioPorCena.append(pgQuestionarioCena)
    pgQuestionario_media.append(pgQuestionarioPorCena)
pgQuestionario_media = pd.DataFrame(pgQuestionario_media, columns=nomeCenas[1:5], index = participantes.index)
pgQuestionario_media = pd.concat([pgQuestionario_media,participantes.loc[:,'Deficiência']],axis = 1)
pgQuestionario_media.rename(columns = {'Deficiência' : 'Visual Impairity'}, inplace=True) 
pgQuestionario_media.index.names = ['Participant']
pgQuestionario_media


,Audio,Haptic Belt,Virtual Cane,Mixture,Visual Impairity
Participant,,,,,
001C,3.333333,3.8,3.2,2.555556,0
001,5.000000,3.4,3.0,3.666667,1


### Média pontuação global guiagem (por cenários, por deficiencia)

In [24]:
pgQuestionario_media_def = pgQuestionario_media.groupby(by = 'Visual Impairity').mean()
pgQuestionario_media_def

,Audio,Haptic Belt,Virtual Cane,Mixture
Visual Impairity,,,,
0,3.333333,3.8,3.2,2.555556
1,5.000000,3.4,3.0,3.666667


# Analisar os dados do NASA-TLX usando ANOVA

In [25]:
# Gerar um boxplot para ver a distribuição dos tratamentos e ver como está o comportamento da variância
plt.figure(figsize=(16, 12))
bp = sns.boxplot(x='Método', hue='Etapa', y='value', data=dimensoesNasa_melt[0], color='#99c2a2')
sp = sns.swarmplot(x="Método", hue='Etapa', y="value", data=dimensoesNasa_melt[0], color='#7d0013')
plt.show()

NameError: name 'dimensoesNasa_melt' is not defined

<Figure size 1152x864 with 0 Axes>

In [ ]:
# stats f_oneway functions takes the groups as input and returns ANOVA F and p value
fvalue, pvalue = stats.f_oneway(dimensoesNasa_pivot[0]['1,1'],dimensoesNasa_pivot[0]['1,2'],dimensoesNasa_pivot[0]['2,1'],dimensoesNasa_pivot[0]['2,2'],dimensoesNasa_pivot[0]['3,1'],dimensoesNasa_pivot[0]['3,2'],dimensoesNasa_pivot[0]['4,1'],dimensoesNasa_pivot[0]['4,2'],dimensoesNasa_pivot[0]['5,1'],dimensoesNasa_pivot[0]['5,2'])
print(fvalue, pvalue)
# 17.492810457516338 2.639241146210922e-05

# Ordinary Least Squares (OLS) model
model = ols('value ~ 4(Método)', data=dimensoesNasa_melt).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table
## output (ANOVA F and p value)
#                sum_sq    df         F    PR(>F)
#C(treatments)  3010.95   3.0  17.49281  0.000026
#Residual        918.00  16.0       NaN       NaN
#
## ANOVA table using bioinfokit v1.0.3 or later (it uses wrapper script for anova_lm)
#from bioinfokit.analys import stat
#res = stat()
#res.anova_stat(df=df_melt, res_var='value', anova_model='value ~ C(treatments)')
#res.anova_summary
## output (ANOVA F and p value)
#                 df   sum_sq   mean_sq         F    PR(>F)
#C(treatments)   3.0  3010.95  1003.650  17.49281  0.000026
#Residual       16.0   918.00    57.375       NaN       NaN
#
## note: if the data is balanced (equal sample size for each group), Type 1, 2, and 3 sums of squares
## (typ parameter) will produce similar results.

0.5903765552292003 0.8018383462480272


<string>:1: SyntaxWarning: 'int' object is not callable; perhaps you missed a comma?


In [ ]:
dimensoesNasa_melt

[    Participante  Método  Etapa Cena Dimensões do NASA  value
 0              1       1      1  1,1    Demanda Mental      5
 1              1       1      2  1,2    Demanda Mental     15
 2              1       2      1  2,1    Demanda Mental      1
 3              1       2      2  2,2    Demanda Mental      9
 4              1       3      1  3,1    Demanda Mental      1
 ..           ...     ...    ...  ...               ...    ...
 95            10       3      2  3,2    Demanda Mental      4
 96            10       4      1  4,1    Demanda Mental      1
 97            10       4      2  4,2    Demanda Mental      1
 98            10       5      1  5,1    Demanda Mental     10
 99            10       5      2  5,2    Demanda Mental     19
 
 [100 rows x 6 columns],
     Participante  Método  Etapa Cena Dimensões do NASA  value
 0              1       1      1  1,1    Demanda física     19
 1              1       1      2  1,2    Demanda física     16
 2              1       2   

In [ ]:
import pandas as pd
# load data file
df = pd.read_csv("https://reneshbedre.github.io/assets/posts/anova/onewayanova.txt", sep="\t")
# reshape the d dataframe suitable for statsmodels package 
df_melt = pd.melt(df.reset_index(), id_vars=['index'], value_vars=['A', 'B', 'C', 'D'])
# replace column names
df_melt.columns = ['index', 'treatments', 'value']
df_melt

,index,treatments,value
0,0,A,25
1,1,A,30
2,2,A,28
3,3,A,36
4,4,A,29
5,0,B,45
6,1,B,55
7,2,B,29
8,3,B,56
9,4,B,40


In [ ]:

# stats f_oneway functions takes the groups as input and returns ANOVA F and p value
fvalue, pvalue = stats.f_oneway(df['A'], df['B'], df['C'], df['D'])
print(fvalue, pvalue)
# 17.492810457516338 2.639241146210922e-05

# get ANOVA table as R like output
import statsmodels.api as sm
from statsmodels.formula.api import ols

# Ordinary Least Squares (OLS) model
model = ols('value ~ C(treatments)', data=df_melt).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table
## output (ANOVA F and p value)
#                sum_sq    df         F    PR(>F)
#C(treatments)  3010.95   3.0  17.49281  0.000026
#Residual        918.00  16.0       NaN       NaN
#
## ANOVA table using bioinfokit v1.0.3 or later (it uses wrapper script for anova_lm)
#from bioinfokit.analys import stat
#res = stat()
#res.anova_stat(df=df_melt, res_var='value', anova_model='value ~ C(treatments)')
#res.anova_summary
## output (ANOVA F and p value)
#                 df   sum_sq   mean_sq         F    PR(>F)
#C(treatments)   3.0  3010.95  1003.650  17.49281  0.000026
#Residual       16.0   918.00    57.375       NaN       NaN
#
## note: if the data is balanced (equal sample size for each group), Type 1, 2, and 3 sums of squares
## (typ parameter) will produce similar results.

17.492810457516338 2.639241146210922e-05


,sum_sq,df,F,PR(>F)
C(treatments),3010.95,3.0,17.49281,0.000026
Residual,918.00,16.0,NaN,NaN


In [ ]:
df_melt

,index,treatments,value
0,0,A,25
1,1,A,30
2,2,A,28
3,3,A,36
4,4,A,29
5,0,B,45
6,1,B,55
7,2,B,29
8,3,B,56
9,4,B,40
